## dataload & FeatureEngineering

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
plt.style.use('seaborn')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import missingno
import pandas as pd
pd.set_option('display.min_rows', 500)
import warnings
warnings.filterwarnings('ignore')

def concat_and_export(train_fe, test_fe):
    
    train_fe['kind']='train'
    test_fe['kind']='test'
    
    df = pd.concat([train_fe,test_fe])
    data_dir = '/opt/ml/input/data' # 경로
    write_path=f'{data_dir}/after_fe_train_test_cumsum.pkl'
    df.to_pickle(write_path,index=False)
    print(f"Write: {write_path}")
    
def export(df, output='after_fe_train_test.pkl'):
    data_dir = '/opt/ml/input/data' # 경로
    write_path=f'{data_dir}/{output}'
    df.to_pickle(write_path)
    print(f"Write: {write_path}")
    
path='../../data/'
train = pd.read_csv(f"{path}/train_data.csv")
test = pd.read_csv(f"{path}/test_data.csv")

day_dict = {'Tuesday': 0,
 'Thursday': 1,
 'Monday': 2,
 'Saturday': 3,
 'Friday': 4,
 'Wednesday': 5,
 'Sunday': 6}

def feature_engineering(df):
    uid2idx = {v:k for k,v in enumerate(sorted(df.userID.unique()))}
    ass2idx = {v:k for k,v in enumerate(sorted(df.assessmentItemID.unique()))}
    test2idx = {v:k for k,v in enumerate(sorted(df.testId.unique()))}

    df2 = df.copy()
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df2.sort_values(by=['userID','Timestamp'], inplace=True)
    
    # userID, assessmentItemID, testId 라벨인코딩
    df2['uidIdx'] = df2.userID.map(uid2idx)
    df2['assIdx'] = df2.assessmentItemID.map(ass2idx)
    df2['testIdx'] = df2.testId.map(test2idx)
    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df2['user_correct_answer'] = df2.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df2['user_total_answer'] = df2.groupby('userID')['answerCode'].cumcount()
    df2['user_acc'] = df2['user_correct_answer']/df2['user_total_answer']
    df2['month'] = pd.to_datetime(df2.Timestamp).dt.month
    df2['day'] = pd.to_datetime(df2.Timestamp).dt.day
    df2['hour'] = pd.to_datetime(df2.Timestamp).dt.hour
    df2['dayname'] = pd.to_datetime(df2.Timestamp).dt.day_name().map(day_dict)
    df2['big_category'] = df2.testId.map(lambda x:x[2]).astype(int)
    df2['problem_num'] = df2.assessmentItemID.map(lambda x: int(x[-3:]))
    df2['mid_category'] = df2.testId.map(lambda x: int(x[-3:]))

    # testId와 KnowledgeTag의 전체 정답률은 한번에 계산
    # 아래 데이터는 제출용 데이터셋에 대해서도 재사용
    correct_t = df2.groupby(['testId'])['answerCode'].agg(['mean', 'std', 'sum'])
    correct_t.columns = ["test_mean", "test_std", 'test_sum']
    correct_k = df2.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'std', 'sum'])
    correct_k.columns = ["tag_mean", 'tag_std', 'tag_sum']

    df2 = pd.merge(df2, correct_t, on=['testId'], how="left")
    df2 = pd.merge(df2, correct_k, on=['KnowledgeTag'], how="left")
    
    # 유저별 문제푼 시간, solvesec_3600, time_category
    df2['Timestamp2'] = pd.to_datetime(df2.Timestamp)
    df2['solvetime'] = df2.groupby('userID')['Timestamp2'].diff().fillna(pd.Timedelta(seconds=0))
    df2['solvesec'] = df2.solvetime.map(lambda x : x.total_seconds())
    df2['solvesec_3600'] = df2.solvesec
    df2.loc[df2.solvesec>=3600,'solvesec_3600']=3600

    df2['time_category'] = ''
    tc = [0,5,7,10,60,600,1200,2400,3600]
    df2.loc[(df2.solvesec==0), 'time_category'] = "0 - [0,0]"
    for i in range(len(tc)-1):
        s,e = tc[i],tc[i+1]
        df2.loc[(df2.solvesec>s) & (df2.solvesec<=e),'time_category']=f"{i+1} - ({s}, {e}]"
    df2.loc[(df2.solvesec>=tc[-1]),'time_category'] = f"{i+2} - ({e}, )"
    timecat2idx={k:v for v,k in enumerate(sorted(df2.time_category.unique()))}
    df2['time_category'] = df2.time_category.map(timecat2idx)
    
    # 유저별 문제푼 시간 Cumsum
    df2['solvesec_cumsum'] = df2.groupby(['userID','testId'])['solvesec_3600'].cumsum()%3601
    df2['solvecumsum_category'] = ''
    tc = [0,5,7,10,60,600,1200,2400,3600,7200]
    df2.loc[(df2.solvesec_cumsum==0), 'solvecumsum_category'] = "0 - [0,0]"
    for i in range(len(tc)-1):
        s,e = tc[i],tc[i+1]
        df2.loc[(df2.solvesec_cumsum>s) & (df2.solvesec_cumsum<=e),'solvecumsum_category']=f"{i+1} - ({s}, {e}]"
    df2.loc[(df2.solvesec_cumsum>=tc[-1]),'solvecumsum_category'] = f"{i+2} - ({e}, )"
    solvecumsum_category2idx={k:v for v,k in enumerate(sorted(df2.solvecumsum_category.unique()))}
    df2['solvecumsum_category'] = df2.solvecumsum_category.map(solvecumsum_category2idx)
    
    ## big_category
    # big_category 전체 정답률, 표준편차
#     big_category_answermean = dict(df2.groupby("big_category").answerCode.mean())
#     big_category_answerstd = dict(df2.groupby("big_category").answerCode.std())
#     df2['big_category_acc'] = df2.big_category.map(big_category_answermean)
#     df2['big_category_std'] = df2.big_category.map(big_category_answerstd)

    # 유저별 big category 문제 푼 횟수, 맞춤 횟수, 누적 정답률
    df2['big_category_cumconut'] = df2.groupby(['userID','big_category']).answerCode.cumcount()
    df2['big_category_answer'] = df2.groupby(['userID','big_category']).answerCode.transform(lambda x: x.cumsum().shift(1)).fillna(0)
    df2['big_category_user_cum_acc'] = (df2['big_category_answer'] / df2['big_category_cumconut']).fillna(0)
    
    # 유저별 정답률, 표준 편차 (마지막제출 제외)
#     df2_user_big_ans = df2[df2.userID == df2.userID.shift(-1)].groupby(['userID','big_category']).answerCode.mean().reset_index()
#     df2_user_big_ans.rename(columns={'answerCode':'big_category_user_acc'},inplace=True)
#     df2 = df2.merge(df2_user_big_ans,on=['userID','big_category'])
    
#     df2_user_big_ans_std = df2[df2.userID == df2.userID.shift(-1)].groupby(['userID','big_category']).answerCode.std().reset_index()
#     df2_user_big_ans_std.rename(columns={'answerCode':'big_category_user_std'},inplace=True)
#     df2 = df2.merge(df2_user_big_ans_std,on=['userID','big_category'])   
    
    ## mid_category
    # mid_category 전체 정답률, 표준편차
#     mid_category_answermean = dict(df2.groupby("mid_category").answerCode.mean())
#     mid_category_answerstd = dict(df2.groupby("mid_category").answerCode.std())
#     df2['mid_category_acc'] = df2.mid_category.map(mid_category_answermean)
#     df2['mid_category_std'] = df2.mid_category.map(mid_category_answerstd)

    # 유저별 mid category 문제 푼 횟수, 맞춤 횟수, 누적 정답률
    df2['mid_category_cumconut'] = df2.groupby(['userID','mid_category']).answerCode.cumcount()
    df2['mid_category_answer'] = df2.groupby(['userID','mid_category']).answerCode.transform(lambda x: x.cumsum().shift(1)).fillna(0)
    df2['mid_category_user_cum_acc'] = (df2['mid_category_answer'] / df2['mid_category_cumconut']).fillna(0)

    # 유저별 정답률, 표준 편차 (마지막제출 제외)
#     df2_user_mid_ans = df2[df2.userID == df2.userID.shift(-1)].groupby(['userID','mid_category']).answerCode.mean().reset_index()
#     df2_user_mid_ans.rename(columns={'answerCode':'mid_category_user_acc'},inplace=True)
#     df2 = df2.merge(df2_user_mid_ans,on=['userID','mid_category'])

#     df2_user_mid_ans_std = df2[df2.userID == df2.userID.shift(-1)].groupby(['userID','mid_category']).answerCode.std().reset_index()
#     df2_user_mid_ans_std.rename(columns={'answerCode':'mid_category_user_std'},inplace=True)
#     df2 = df2.merge(df2_user_mid_ans_std,on=['userID','mid_category']) 


    ### 문항별 난이도
    ass_acc_dict = dict(df2[df2.answerCode!=-1].groupby('assessmentItemID').answerCode.mean())
    df2['ass_acc_mean'] = df2.assessmentItemID.map(ass_acc_dict)
    df2['ass_difficulty'] = 1 - df2['ass_acc_mean']
    
    ass_acc_std_dict = dict(df2[df2.answerCode!=-1].groupby('assessmentItemID').answerCode.std())
    df2['ass_acc_std'] = df2.assessmentItemID.map(ass_acc_std_dict)

    ### 문제 번호별 난이도
    pb_num_dict = dict(df2[df2.answerCode!=-1].groupby('problem_num').answerCode.mean())
    df2['pb_num_acc_mean'] = df2.problem_num.map(pb_num_dict)
    df2['pb_num_difficulty'] = 1 - df2['pb_num_acc_mean']
    
    pb_num_std_dict = dict(df2[df2.answerCode!=-1].groupby('problem_num').answerCode.std())
    df2['pb_num_acc_std'] = df2.problem_num.map(pb_num_std_dict)
    
    ## assess_count
    df2['assess_count'] = df2.groupby(['userID','assessmentItemID']).answerCode.cumcount()
    
    df2.sort_values(by=['userID','Timestamp'], inplace=True)
    return df2

train['kind']='train'
test['kind']='test'
df = pd.concat([train,test])
df = feature_engineering(df)
df2=df.copy()

In [2]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2526700 entries, 0 to 2526699
Data columns (total 45 columns):
 #   Column                     Dtype          
---  ------                     -----          
 0   userID                     int64          
 1   assessmentItemID           object         
 2   testId                     object         
 3   answerCode                 int64          
 4   Timestamp                  object         
 5   KnowledgeTag               int64          
 6   kind                       object         
 7   uidIdx                     int64          
 8   assIdx                     int64          
 9   testIdx                    int64          
 10  user_correct_answer        float64        
 11  user_total_answer          int64          
 12  user_acc                   float64        
 13  month                      int64          
 14  day                        int64          
 15  hour                       int64          
 16  dayname           

## assess_count

In [12]:
def assess_count(data):
    data['cnt'] = 1
    data['assess_count'] = data.groupby(['userID', 'assessmentItemID'])['cnt'].cumsum()
    return data.drop(columns = 'cnt')

In [13]:
df2 = assess_count(df2)

In [55]:
df2['assess_count'] = df2.groupby(['userID','assessmentItemID']).answerCode.cumcount()

## 과거에 해당 문제 몇번 맞혔는지

In [4]:
df2['shift'] = df2.groupby(['userID','assessmentItemID']).answerCode.shift().fillna(0)

In [5]:
df2['past_ass_correct'] = df2.groupby(['userID','assessmentItemID'])['shift'].cumsum()

## Timediff

In [16]:
import time

In [23]:
def percentile(s):
    return np.sum(s) / len(s)

def time_feature_engineering(df):
    df['Time'] = df['Timestamp2'].apply(lambda x: int(time.mktime(x.timetuple())))
    df['Timediff'] = df.groupby(['userID','testId','assess_count'])['Time'].diff()
    df['Timediff'] = df['Timediff'].fillna(0)
    df['Timediff'] = df['Timediff'].apply(lambda x : x if x < 650 else 650)
    df['Timepassed'] = df.groupby(['userID','testId','assess_count'])['Timediff'].cumsum()
    timediff = df.groupby('Timediff').agg({'answerCode':percentile}).reset_index()
    timediff.columns =['Timediff','Time_answer_rate']
    df = df.merge(timediff,how='left',on='Timediff')

    return df

In [24]:
df2 = time_feature_engineering(df2)

## timediff category, cumsum, cumsum_category

In [34]:
df2[df2.timediff_category==''].iloc[:,-10:]

,pb_num_difficulty,pb_num_acc_std,shift,past_ass_correct,assess_count,Time,Timediff,Timepassed,Time_answer_rate,timediff_category


In [32]:
df2['timediff_category'] = ''
tc = [0,5,7,10,60,120,180,240,300,360,480]
df2.loc[(df2.Timediff==0), 'timediff_category'] = "0 - [0,0]"
for i in range(len(tc)-1):
    s,e = tc[i],tc[i+1]
    df2.loc[(df2.Timediff>s) & (df2.Timediff<=e),'timediff_category']=f"{i+1} - ({s}, {e}]"
df2.loc[(df2.Timediff>=tc[-1]),'timediff_category'] = f"{i+2} - ({e}, )"

In [35]:
time_grp = df2.groupby('timediff_category')['answerCode'].agg(['mean','count'])
time_grp['C'] = time_grp.index.str.extract(r'(\d+)').astype(int)[0].tolist()
time_grp.sort_values('C')[['mean','count']]

,mean,count
timediff_category,,
"0 - [0,0]",0.744570,423545
"1 - (0, 5]",0.245650,364502
"2 - (5, 7]",0.490895,69080
"3 - (7, 10]",0.664487,102023
"4 - (10, 60]",0.751337,1001012
"5 - (60, 120]",0.710219,291210
"6 - (120, 180]",0.692268,106518
"7 - (180, 240]",0.686698,51369
"8 - (240, 300]",0.677666,28849


In [36]:
timediffcat2idx={k:v for v,k in enumerate(sorted(df2.timediff_category.unique()))}
df2['timediff_category'] = df2.time_category.map(timediffcat2idx)

In [37]:
df2.Timepassed.describe().T

count    2.526700e+06
mean     1.776232e+02
std      2.949966e+02
min      0.000000e+00
25%      1.100000e+01
50%      6.700000e+01
75%      2.030000e+02
max      5.151000e+03
Name: Timepassed, dtype: float64

In [41]:
# 유저별 문제푼 시간 Cumsum
# df2['solvesec_cumsum'] = df2.groupby(['userID','testId'])['solvesec_3600'].cumsum()%3601
df2['timepassed_category'] = ''
tc = [0,5,7,10,60,600,2400,3600,7200]
df2.loc[(df2.solvesec_cumsum==0), 'timepassed_category'] = "0 - [0,0]"
for i in range(len(tc)-1):
    s,e = tc[i],tc[i+1]
    df2.loc[(df2.solvesec_cumsum>s) & (df2.solvesec_cumsum<=e),'timepassed_category']=f"{i+1} - ({s}, {e}]"
df2.loc[(df2.solvesec_cumsum>=tc[-1]),'timepassed_category'] = f"{i+2} - ({e}, )"

In [42]:
timepassed_grp = df2.groupby('timepassed_category')['answerCode'].agg(['mean','count'])
timepassed_grp['C'] = timepassed_grp.index.str.extract(r'(\d+)').astype(int)[0].tolist()
timepassed_grp.sort_values('C')[['mean','count']]

,mean,count
timepassed_category,,
"0 - [0,0]",0.479131,14735
"1 - (0, 5]",0.275780,90895
"2 - (5, 7]",0.395883,40224
"3 - (7, 10]",0.442350,59003
"4 - (10, 60]",0.646462,515006
"5 - (60, 600]",0.684740,1129444
"6 - (600, 2400]",0.644952,277422
"7 - (2400, 3600]",0.733706,399971


In [43]:
timepassed_category2idx={k:v for v,k in enumerate(sorted(df2.timepassed_category.unique()))}
df2['timepassed_category'] = df2.timepassed_category.map(timepassed_category2idx)

In [44]:
df2.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'kind', 'uidIdx', 'assIdx', 'testIdx',
       'user_correct_answer', 'user_total_answer', 'user_acc', 'month', 'day',
       'hour', 'dayname', 'big_category', 'problem_num', 'mid_category',
       'test_mean', 'test_std', 'test_sum', 'tag_mean', 'tag_std', 'tag_sum',
       'Timestamp2', 'solvetime', 'solvesec', 'solvesec_3600', 'time_category',
       'solvesec_cumsum', 'solvecumsum_category', 'big_category_cumconut',
       'big_category_answer', 'big_category_user_cum_acc',
       'mid_category_cumconut', 'mid_category_answer',
       'mid_category_user_cum_acc', 'ass_acc_mean', 'ass_difficulty',
       'ass_acc_std', 'pb_num_acc_mean', 'pb_num_difficulty', 'pb_num_acc_std',
       'shift', 'past_ass_correct', 'assess_count', 'Time', 'Timediff',
       'Timepassed', 'Time_answer_rate', 'timediff_category',
       'timepassed_category'],
      dtype='object')

In [60]:
df2.assess_count.value_counts()

0    2476706
1      49100
2        894
Name: assess_count, dtype: int64

In [61]:
df2.past_ass_correct.value_counts()

0.0    2492923
1.0      33413
2.0        364
Name: past_ass_correct, dtype: int64

In [62]:
df2[df2.past_ass_correct==2]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,kind,uidIdx,assIdx,testIdx,...,pb_num_acc_std,shift,past_ass_correct,assess_count,Time,Timediff,Timepassed,Time_answer_rate,timediff_category,timepassed_category
141393,165,A090007002,A090000007,0,2020-04-23 01:11:13,4725,train,165,8984,1469,...,0.448847,1.0,2.0,2,1587571873,4.0,4.0,0.246453,NaN,5
141395,165,A090007004,A090000007,0,2020-04-23 01:11:27,4697,train,165,8986,1469,...,0.472479,1.0,2.0,2,1587571887,3.0,18.0,0.228741,NaN,5
141406,165,A090009001,A090000009,1,2020-04-28 01:19:28,5141,train,165,8995,1471,...,0.433190,1.0,2.0,2,1588004368,0.0,0.0,0.744570,NaN,5
141407,165,A090009002,A090000009,1,2020-04-28 01:19:54,10741,train,165,8996,1471,...,0.448847,1.0,2.0,2,1588004394,26.0,26.0,0.767259,NaN,5
141417,165,A090011004,A090000011,0,2020-04-29 23:30:32,9728,train,165,9014,1473,...,0.472479,1.0,2.0,2,1588170632,4.0,16.0,0.246453,NaN,6
141418,165,A090011005,A090000011,0,2020-04-29 23:30:41,9728,train,165,9015,1473,...,0.490026,1.0,2.0,2,1588170641,9.0,25.0,0.670500,NaN,6
141432,165,A090013005,A090000013,0,2020-05-04 21:30:46,5261,train,165,9031,1475,...,0.490026,1.0,2.0,2,1588595446,3.0,13.0,0.228741,NaN,5
141435,165,A090013008,A090000013,0,2020-05-04 21:30:54,4210,train,165,9034,1475,...,0.498037,1.0,2.0,2,1588595454,3.0,21.0,0.228741,NaN,5
141440,165,A090014005,A090000014,1,2020-05-06 23:18:07,9728,train,165,9039,1476,...,0.490026,1.0,2.0,2,1588774687,9.0,29.0,0.670500,NaN,5
141456,165,A090015001,A090000015,0,2020-05-11 22:43:08,4210,train,165,9041,1477,...,0.433190,1.0,2.0,2,1589204588,0.0,0.0,0.744570,NaN,6


In [48]:
cond1 = df2.kind=='test'
cond2 = df2.userID != df2.userID.shift(-7)
cond3 = df2.past_ass_correct!=0

In [59]:
base = ['userID', 'assessmentItemID', 'answerCode', 'Timestamp','KnowledgeTag']
add_features = ['assess_count','past_ass_correct']
cols = base + add_features
df2[cond1 & cond2 & cond3][cols]

,userID,assessmentItemID,answerCode,Timestamp,KnowledgeTag,assess_count,past_ass_correct
171147,203,A060073001,1,2020-10-23 09:47:03,616,1,1.0
171148,203,A060073002,1,2020-10-23 09:50:02,616,1,1.0
171149,203,A060073003,1,2020-10-23 09:50:34,616,1,1.0
171150,203,A060073004,1,2020-10-23 09:52:39,616,1,1.0
171151,203,A060073005,1,2020-10-23 09:55:21,616,1,1.0
171152,203,A060073006,1,2020-10-23 09:58:49,616,1,1.0
171153,203,A060073007,-1,2020-10-23 10:00:55,616,1,1.0
605504,765,A010157004,0,2020-10-22 07:41:46,7807,1,1.0
605505,765,A010157005,1,2020-10-22 07:42:18,7807,1,1.0
963237,1268,A010133004,1,2020-10-24 01:43:05,5845,1,1.0


In [57]:
df2[(df2.userID==7329)][cols]

,userID,assessmentItemID,answerCode,Timestamp,KnowledgeTag,assess_count,past_ass_correct
2524735,7329,A040198001,0,2020-01-02 12:50:41,2133,0,0.0
2524736,7329,A040198002,1,2020-01-02 12:50:46,2133,0,0.0
2524737,7329,A040198003,1,2020-01-02 12:50:56,2133,0,0.0
2524738,7329,A040198004,1,2020-01-02 12:51:04,2133,0,0.0
2524739,7329,A010014001,1,2020-01-05 03:56:54,5834,0,0.0
2524740,7329,A010014002,1,2020-01-05 03:57:18,5834,0,0.0
2524741,7329,A010014003,1,2020-01-05 03:57:36,5834,0,0.0
2524742,7329,A010014004,1,2020-01-05 03:58:20,5834,0,0.0
2524743,7329,A010014005,1,2020-01-05 03:58:39,5834,0,0.0
2524744,7329,A040196001,1,2020-10-07 10:30:54,2131,0,0.0


## 내보내기

In [45]:
df2.sort_values(by=['userID','Timestamp'], inplace=True)

In [63]:
export(df2, output='lgbm_assess_count_solvesec_650.pkl')

Write: /opt/ml/input/data/lgbm_assess_count_solvesec_650.pkl
